# Entendendo Perceptrons Multicamadas (MLPs)

In [ ]:
# Bibliotecas 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

## Exercício 1

**Dados do problema.**
Entrada $x=[0.5,\,-0.2]$, alvo $y=1.0$.
Camada oculta (2 neurônios, tanh):

$$
W^{(1)}=\begin{bmatrix}0.3&-0.1\\ 0.2&0.4\end{bmatrix},\quad
b^{(1)}=\begin{bmatrix}0.1\\-0.2\end{bmatrix}
$$

Saída (1 neurônio, tanh):

$$
W^{(2)}=\begin{bmatrix}0.5&-0.3\end{bmatrix},\quad
b^{(2)}=0.2
$$

---

### 1) *Forward pass*

**Pré-ativações na oculta** $z^{(1)}=W^{(1)}x+b^{(1)}$:

$$
\begin{aligned}
z^{(1)}_1&=0.3\cdot0.5+(-0.1)\cdot(-0.2)+0.1=0.27 \\
z^{(1)}_2&=0.2\cdot0.5+0.4\cdot(-0.2)-0.2=-0.18
\end{aligned}
\Longrightarrow\quad
z^{(1)}=\begin{bmatrix}0.270000\\-0.180000\end{bmatrix}
$$

**Ativações na oculta** $a^{(1)}=\tanh(z^{(1)})$:

$$
a^{(1)}=\begin{bmatrix}\tanh(0.27)\\ \tanh(-0.18)\end{bmatrix}
=\begin{bmatrix}0.263625\\-0.178081\end{bmatrix}
$$

**Pré-ativação na saída** $z^{(2)}=W^{(2)}a^{(1)}+b^{(2)}$:

$$
z^{(2)}=0.5\cdot0.263625+(-0.3)\cdot(-0.178081)+0.2
=0.385237
$$

**Saída** $\hat y=\tanh(z^{(2)})=\tanh(0.385237)=\mathbf{0.367247}$.

---

### 2) *Loss*

$$
L=(y-\hat y)^2=(1-0.367247)^2=\mathbf{0.400377}.
$$

---

### 3) *Backward pass*

Derivada da perda em relação à saída:

$$
\frac{\partial L}{\partial \hat y}=2(\hat y-y)=2(0.367247-1)=-1.265507.
$$

Derivada da tanh: $\frac{d}{dz}\tanh(z)=1-\tanh^2(z)$. Logo:

$$
\frac{\partial L}{\partial z^{(2)}}=\frac{\partial L}{\partial \hat y}\,(1-\hat y^2)
=-1.265507\cdot(1-0.367247^2)=\mathbf{-1.094828}.
$$

**Gradientes da camada de saída**

$$
\frac{\partial L}{\partial W^{(2)}}=\frac{\partial L}{\partial z^{(2)}}\,a^{(1)}
=\begin{bmatrix}-0.288624 & 0.194968\end{bmatrix},\quad
\frac{\partial L}{\partial b^{(2)}}=\mathbf{-1.094828}.
$$

**Propagação para a oculta**

$$
\frac{\partial L}{\partial a^{(1)}}=\frac{\partial L}{\partial z^{(2)}}\,W^{(2)}
=\begin{bmatrix}-0.547414\\ 0.328448\end{bmatrix},\qquad
1-(a^{(1)})^2=\begin{bmatrix}0.930502\\ 0.968287\end{bmatrix}.
$$

$$
\frac{\partial L}{\partial z^{(1)}}=\frac{\partial L}{\partial a^{(1)}}\odot\bigl(1-(a^{(1)})^2\bigr)
=\begin{bmatrix}-0.509370\\ 0.318032\end{bmatrix}.
$$

**Gradientes da camada oculta** (produto externo com $x=[0.5,-0.2]$):

$$
\frac{\partial L}{\partial W^{(1)}}
=\begin{bmatrix}
-0.254685 & 0.101874\\
\phantom{-}0.159016 & -0.063606
\end{bmatrix},\quad
\frac{\partial L}{\partial b^{(1)}}=\begin{bmatrix}-0.509370\\ 0.318032\end{bmatrix}.
$$

---

### 4) *Atualização de parâmetros*  $(\eta=\mathbf{0.1})$

$$
\begin{aligned}
W^{(2)}_{\text{novo}}&=W^{(2)}-\eta\,\frac{\partial L}{\partial W^{(2)}} 
=\begin{bmatrix}0.528862 & -0.319497\end{bmatrix} \\
b^{(2)}_{\text{novo}}&=b^{(2)}-\eta\,\frac{\partial L}{\partial b^{(2)}}
=\mathbf{0.309483} \\
W^{(1)}_{\text{novo}}&=W^{(1)}-\eta\,\frac{\partial L}{\partial W^{(1)}} \\
&=\begin{bmatrix}
0.325468 & -0.110187\\
0.184098 & \phantom{-}0.406361
\end{bmatrix}\\[2pt]
b^{(1)}_{\text{novo}}&=b^{(1)}-\eta\,\frac{\partial L}{\partial b^{(1)}}
=\begin{bmatrix}\phantom{-}0.150937\\ -0.231803\end{bmatrix}.
\end{aligned}
$$

## Exercício 2 — MLP *from scratch* (classificação binária em 2D)

Gerar um dataset 2D com **1 cluster** para a classe 0 e **2 clusters** para a classe 1 (usando `make_classification` em subconjuntos), treinar um **MLP do zero** (NumPy apenas) com 1 camada oculta, loss binário (BCE), e avaliar: perda de treino, acurácia de teste e fronteira de decisão.

In [ ]:
np.random.seed(42)

n0, n1 = 500, 500  # total=1000

# Subconjunto só com classe 0 (1 cluster)
X0, y0 = make_classification(
    n_samples=n0, n_features=2, n_informative=2, n_redundant=0,
    n_clusters_per_class=1, n_classes=2, weights=[1.0, 0.0],
    class_sep=1.5, flip_y=0.0, random_state=42
)

# Subconjunto só com classe 1 (2 clusters)
X1, y1 = make_classification(
    n_samples=n1, n_features=2, n_informative=2, n_redundant=0,
    n_clusters_per_class=2, n_classes=2, weights=[0.0, 1.0],
    class_sep=1.5, flip_y=0.0, random_state=43
)

# Junta e embaralha
X = np.vstack([X0, X1])
y = np.hstack([y0, y1])  # rótulos {0,1}

perm = np.random.permutation(len(X))
X, y = X[perm], y[perm]

# (segurança) se por acaso seus rótulos estiverem em {-1,+1}, converte para {0,1}
if set(np.unique(y)) == {-1, 1}:
    y = ((y + 1) // 2).astype(int)

print(X.shape, y.shape, np.bincount(y))


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Opcional (recomendado): padronizar usando estatísticas do treino
mu = X_train.mean(axis=0)
sigma = X_train.std(axis=0) + 1e-8
X_train = (X_train - mu) / sigma
X_test  = (X_test  - mu) / sigma

print("train:", X_train.shape, "test:", X_test.shape)


In [ ]:
# --------- Ativações ---------
def tanh(z):
    return np.tanh(z)

def dtanh(a):
    # derivada em termos da ativação a = tanh(z)
    return 1.0 - a**2

def relu(z):
    return np.maximum(0.0, z)

def drelu(z):
    return (z > 0.0).astype(z.dtype)

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def softmax(z):
    # estável numericamente
    z = z - z.max(axis=1, keepdims=True)
    e = np.exp(z)
    return e / e.sum(axis=1, keepdims=True)

# --------- Perdas ---------
def bce_loss(y_true, y_prob, eps=1e-9):
    # y_true shape: (N,1) com {0,1}; y_prob shape: (N,1)
    y_prob = np.clip(y_prob, eps, 1.0 - eps)
    return -(y_true*np.log(y_prob) + (1-y_true)*np.log(1-y_prob)).mean()

def ce_loss(y_true_onehot, y_prob, eps=1e-9):
    # y_true_onehot shape: (N,C); y_prob shape: (N,C)
    y_prob = np.clip(y_prob, eps, 1.0 - eps)
    return -(y_true_onehot * np.log(y_prob)).sum(axis=1).mean()

# --------- Utilitários ---------
def one_hot(y, num_classes=None):
    y = y.astype(int).ravel()
    if num_classes is None:
        num_classes = int(y.max()) + 1
    out = np.zeros((y.shape[0], num_classes), dtype=float)
    out[np.arange(y.shape[0]), y] = 1.0
    return out

def xavier_limit(fan_in, fan_out):
    return np.sqrt(6.0 / (fan_in + fan_out))

def he_limit(fan_in):
    # He uniform
    return np.sqrt(6.0 / fan_in)


In [ ]:
class MLP:
    """
    MLP genérico (NumPy puro).
    - layer_sizes: lista com dimensões [in, h1, ..., hK, out]
    - activations: lista de strings para cada camada oculta + saída (len = len(layer_sizes)-1)
        opções: 'tanh', 'relu', 'sigmoid', 'softmax' (use 'sigmoid' p/ binário e 'softmax' p/ multiclasse)
    - loss: 'bce' (binário; requer saída 'sigmoid') ou 'ce' (multiclasse; requer saída 'softmax')
    - l2: regularização L2 (lambda), default 0.0
    - lr: taxa de aprendizado
    - seed: reprodutibilidade
    """
    def __init__(self, layer_sizes, activations, loss, lr=0.05, l2=0.0, seed=42):
        assert len(layer_sizes) >= 2, "Precisa de pelo menos entrada e saída"
        assert len(activations) == len(layer_sizes) - 1, "Uma ativação por camada"
        self.sizes = list(layer_sizes)
        self.acts = list(activations)
        self.loss_name = loss
        self.lr = lr
        self.l2 = float(l2)

        self.rng = np.random.default_rng(seed)
        self.params = self._init_params()
        self.loss_hist = []

        # mapear nomes para funções
        self._act = {
            'tanh': (tanh, dtanh),
            'relu': (relu, None),     # derivative usa Z
            'sigmoid': (sigmoid, None), # derivative calculada via BCE no topo; não usada nas ocultas
            'softmax': (softmax, None)
        }

        if loss == 'bce':
            assert self.acts[-1] == 'sigmoid', "BCE requer saída 'sigmoid'"
        elif loss == 'ce':
            assert self.acts[-1] == 'softmax', "CE requer saída 'softmax'"
        else:
            raise ValueError("loss deve ser 'bce' ou 'ce'")

    def _init_params(self):
        params = []
        for l in range(len(self.sizes) - 1):
            fan_in, fan_out = self.sizes[l], self.sizes[l+1]
            act = self.acts[l]
            if act in ('tanh', 'sigmoid', 'softmax'):
                lim = xavier_limit(fan_in, fan_out)
            elif act == 'relu':
                lim = he_limit(fan_in)
            else:
                lim = xavier_limit(fan_in, fan_out)

            W = self.rng.uniform(-lim, lim, size=(fan_in, fan_out))
            b = np.zeros((1, fan_out))
            params.append({'W': W, 'b': b})
        return params

    def _forward(self, X):
        """
        Retorna A_list, Z_list:
        A_list[0] = X
        para l>=1: Z_list[l] = A_list[l-1] @ W_l + b_l; A_list[l] = act(Z_list[l])
        """
        A_list = [X]
        Z_list = [None]
        for l, layer in enumerate(self.params, start=1):
            W, b = layer['W'], layer['b']
            Z = A_list[-1] @ W + b
            act_name = self.acts[l-1]
            if act_name == 'tanh':
                A = tanh(Z)
            elif act_name == 'relu':
                A = relu(Z)
            elif act_name == 'sigmoid':
                A = sigmoid(Z)
            elif act_name == 'softmax':
                A = softmax(Z)
            else:
                raise ValueError(f"Ativação desconhecida: {act_name}")
            Z_list.append(Z); A_list.append(A)
        return A_list, Z_list

    def _compute_loss(self, y_true, Aout):
        if self.loss_name == 'bce':
            loss = bce_loss(y_true.reshape(-1,1), Aout)
        else:  # 'ce'
            if Aout.ndim == 1 or Aout.shape[1] == 1:
                raise ValueError("CE requer probabilidades (N,C) e rótulos one-hot (N,C)")
            loss = ce_loss(y_true, Aout)

        # L2
        if self.l2 > 0.0:
            l2_sum = sum((layer['W']**2).sum() for layer in self.params)
            loss = loss + 0.5 * self.l2 * l2_sum / y_true.shape[0]
        return float(loss)

    def _backward(self, A_list, Z_list, y_true):
        grads = [None] * len(self.params)
        N = A_list[0].shape[0]

        # dZ da camada de saída
        Aout = A_list[-1]
        if self.loss_name == 'bce':
            # BCE + sigmoid => dZ = (Aout - y)/N
            y = y_true.reshape(-1,1)
            dZ = (Aout - y) / N
        else:  # 'ce' + softmax => dZ = (Aout - Y)/N
            Y = y_true
            dZ = (Aout - Y) / N

        # camadas de trás para frente
        for l in reversed(range(len(self.params))):
            A_prev = A_list[l]
            W = self.params[l]['W']

            dW = A_prev.T @ dZ
            db = dZ.sum(axis=0, keepdims=True)

            # L2
            if self.l2 > 0.0:
                dW = dW + self.l2 * W / N

            grads[l] = {'dW': dW, 'db': db}

            if l > 0:  # propagar para trás se não for a primeira camada
                dA_prev = dZ @ W.T
                act_name = self.acts[l-1]  # ativação da camada l
                if act_name == 'tanh':
                    dZ = dA_prev * dtanh(A_list[l])
                elif act_name == 'relu':
                    dZ = dA_prev * drelu(Z_list[l])
                elif act_name in ('sigmoid', 'softmax'):
                    # não usamos sigmoid/softmax em ocultas neste design; se usar, trate aqui:
                    a = A_list[l]
                    if act_name == 'sigmoid':
                        dZ = dA_prev * a * (1 - a)
                    else:
                        raise ValueError("Softmax em camada oculta não suportado")
                else:
                    raise ValueError(f"Ativação desconhecida: {act_name}")
        return grads

    def _step(self, grads):
        for layer, g in zip(self.params, grads):
            layer['W'] -= self.lr * g['dW']
            layer['b'] -= self.lr * g['db']

    def fit(self, X, y, epochs=300, verbose=False):
        """
        y:
          - binário/BCE: array (N,) ou (N,1) com {0,1}
          - CE: one-hot (N,C)
        """
        self.loss_hist.clear()
        for ep in range(1, epochs+1):
            A_list, Z_list = self._forward(X)
            loss = self._compute_loss(y, A_list[-1])
            self.loss_hist.append(loss)
            grads = self._backward(A_list, Z_list, y)
            self._step(grads)
            if verbose and (ep % 50 == 0 or ep == 1):
                print(f"época {ep:03d} | loss={loss:.4f}")
        return self

    def predict_proba(self, X):
        A_list, _ = self._forward(X)
        return A_list[-1]

    def predict(self, X, thr=0.5):
        proba = self.predict_proba(X)
        # BCE binário
        if self.loss_name == 'bce':
            return (proba >= thr).astype(int).ravel()
        # CE multiclasse
        return np.argmax(proba, axis=1)


In [ ]:
# Exemplo binário: 2 -> 8 -> 1, tanh + sigmoid, BCE
mlp_bin = MLP(layer_sizes=[2, 8, 1],
              activations=['tanh', 'sigmoid'],
              loss='bce',
              lr=0.05, l2=0.0, seed=42)

mlp_bin.fit(X_train, y_train, epochs=300, verbose=True)
y_pred = mlp_bin.predict(X_test)
acc = (y_pred == y_test).mean()
print(f"Acurácia (binário): {acc:.3f}")

In [ ]:
def plot_decision_boundary_model(model, X, y, title="Fronteira de decisão"):
    x_min, x_max = X[:,0].min()-1, X[:,0].max()+1
    y_min, y_max = X[:,1].min()-1, X[:,1].max()+1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                         np.linspace(y_min, y_max, 300))
    grid = np.c_[xx.ravel(), yy.ravel()]
    proba = model.predict_proba(grid)
    if proba.ndim == 1 or proba.shape[1] == 1:
        zz = proba.reshape(xx.shape)
    else:
        zz = proba.max(axis=1).reshape(xx.shape)

    plt.figure(figsize=(6,5))
    cs = plt.contourf(xx, yy, zz, levels=20, alpha=0.4)
    plt.colorbar(cs)
    # pontos
    if y.ndim == 2 and y.shape[1] > 1:
        y_plot = np.argmax(y, axis=1)
    else:
        y_plot = y.ravel()
    for cls in np.unique(y_plot):
        pts = X[y_plot==cls]
        plt.scatter(pts[:,0], pts[:,1], s=12, label=f"classe {cls}")
    plt.title(title); plt.xlabel("x1"); plt.ylabel("x2"); plt.legend()
    plt.show()


In [ ]:
plot_decision_boundary_model(mlp_bin, X_train, y_train, title="Fronteira de decisão (treino)")

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(range(1, len(mlp_bin.loss_hist)+1), mlp_bin.loss_hist, marker="o", ms=3)
plt.xlabel("Época"); plt.ylabel("Loss (treino)")
plt.title("MLP — curva de perda (BCE)")
plt.grid(True)
plt.show()


In [ ]:
def confusion_matrix_bin(y_true, y_pred):
    tn = np.sum((y_true==0) & (y_pred==0))
    fp = np.sum((y_true==0) & (y_pred==1))
    fn = np.sum((y_true==1) & (y_pred==0))
    tp = np.sum((y_true==1) & (y_pred==1))
    return np.array([[tn, fp],
                     [fn, tp]])

y_pred_tr = mlp_bin.predict(X_train)
y_pred_te = mlp_bin.predict(X_test)

acc_tr = (y_pred_tr == y_train).mean()
acc_te = (y_pred_te == y_test).mean()
cm_te  = confusion_matrix_bin(y_test, y_pred_te)

print(f"Acurácia (treino): {acc_tr:.3f}")
print(f"Acurácia (teste) : {acc_te:.3f}")
print("Matriz de confusão (teste) [[TN, FP],[FN, TP]]:\n", cm_te)


In [ ]:
def plot_boundary_with_errors(model, X, y, title="Fronteira (teste) + erros"):
    x_min, x_max = X[:,0].min()-1, X[:,0].max()+1
    y_min, y_max = X[:,1].min()-1, X[:,1].max()+1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                         np.linspace(y_min, y_max, 300))
    grid = np.c_[xx.ravel(), yy.ravel()]
    zz = model.predict_proba(grid).reshape(xx.shape)

    plt.figure(figsize=(6,5))
    cs = plt.contourf(xx, yy, zz, levels=20, alpha=0.35)
    plt.colorbar(cs)

    y_hat = model.predict(X)
    err = y_hat != y

    for cls in np.unique(y):
        pts = X[(y==cls) & (~err)]
        plt.scatter(pts[:,0], pts[:,1], s=12, label=f"classe {cls}")

    if err.any():
        plt.scatter(X[err,0], X[err,1], s=40, facecolors='none', edgecolors='r', linewidths=1.2, label="erros")

    plt.title(title); plt.xlabel("x1"); plt.ylabel("x2"); plt.legend()
    plt.show()

plot_boundary_with_errors(mlp_bin, X_test, y_test, title="Fronteira de decisão (teste) + erros")


O MLP de 1 camada oculta aprendeu uma **fronteira não linear** que separa razoavelmente o único cluster da classe 0 dos **dois clusters** da classe 1. A **perda caiu** de \~0.62 para \~0.52 e a **acurácia de teste** ficou \~0.62, compatível com a **sobreposição** visível entre as classes: há regiões onde ambos os rótulos são plausíveis, e uma fronteira suave (tanh + sigmoid) não resolve todas. Ainda assim, o modelo captura a estrutura multimodal, melhor que um classificador linear. Ganhos adicionais viriam de mais capacidade (mais neurônios/camadas), regularização e ajuste de taxa de aprendizado/épocas.


## Exercício 3 — MLP multiclasse (3 classes, 4 features)

Gerar 1500 amostras, 3 classes, 4 features; 2 clusters para a classe 0, 3 para a classe 1 e 4 para a classe 2 (combinando subconjuntos). Treinar a **mesma MLP** do Ex. 2 (código idêntico), trocando apenas saída e função de perda.


In [ ]:
rng = np.random.default_rng(42)

# 1500 = 500 por classe
n0 = n1 = n2 = 500
F = 4

# Classe 0: 2 clusters
X0, y0 = make_classification(
    n_samples=n0, n_features=F, n_informative=F, n_redundant=0,
    n_classes=2, n_clusters_per_class=2, weights=[1.0, 0.0],
    class_sep=1.6, flip_y=0.0, random_state=10
)
y0[:] = 0

# Classe 1: 3 clusters
X1, y1 = make_classification(
    n_samples=n1, n_features=F, n_informative=F, n_redundant=0,
    n_classes=2, n_clusters_per_class=3, weights=[0.0, 1.0],
    class_sep=1.5, flip_y=0.0, random_state=11
)
y1[:] = 1

# Classe 2: 4 clusters
X2, y2 = make_classification(
    n_samples=n2, n_features=F, n_informative=F, n_redundant=0,
    n_classes=2, n_clusters_per_class=4, weights=[0.0, 1.0],
    class_sep=1.4, flip_y=0.0, random_state=12
)
y2[:] = 2

X = np.vstack([X0, X1, X2])
y = np.concatenate([y0, y1, y2]).astype(int)

perm = rng.permutation(len(X))
X, y = X[perm], y[perm]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# padronização por estatística do treino
mu, sig = X_train.mean(axis=0), X_train.std(axis=0) + 1e-8
X_train = (X_train - mu) / sig
X_test  = (X_test  - mu) / sig


In [ ]:
# Saída com 3 neurônios, tanh nas ocultas, softmax na saída, CE
mlp_mc = MLP(layer_sizes=[X_train.shape[1], 16, 3],
             activations=['tanh', 'softmax'],
             loss='ce', lr=0.05, l2=0.0, seed=42)

# rótulos one-hot para CE
def one_hot(y, C=None):
    C = int(y.max())+1 if C is None else C
    Y = np.zeros((y.size, C)); Y[np.arange(y.size), y] = 1.0
    return Y

Y_train = one_hot(y_train, 3)
Y_test  = one_hot(y_test, 3)

mlp_mc.fit(X_train, Y_train, epochs=400, verbose=True)

y_pred = mlp_mc.predict(X_test)          # argmax
acc = (y_pred == y_test).mean()
print(f"Acurácia (teste, 3 classes): {acc:.3f}")


In [ ]:
plt.plot(mlp_mc.loss_hist); plt.xlabel("época"); plt.ylabel("loss (CE)")
plt.title("Curva de treino — CE"); plt.show()

Z = PCA(n_components=2, random_state=0).fit_transform(X_test)
y_hat = mlp_mc.predict(X_test)
for c in np.unique(y_test):
    pts = Z[y_test==c]
    plt.scatter(pts[:,0], pts[:,1], s=12, label=f"true {c}")
plt.scatter(Z[:,0], Z[:,1], c=y_hat, s=8, cmap="tab10", alpha=0.25, label="pred")
plt.legend(); plt.title("Teste (PCA 2D): rótulo real vs. predito"); plt.show()


O modelo com uma única camada oculta conseguiu reduzir a perda de **1.34 → 0.68** ao longo de 400 épocas, mostrando aprendizado consistente. A curva de treino apresenta decaimento suave, mas ainda relativamente lento, indicando que a capacidade de representação é limitada.

No teste, a acurácia atingiu **72,3%** em um problema de 3 classes. O gráfico de PCA revela sobreposição considerável entre os rótulos verdadeiros e preditos, evidenciando dificuldade em separar regiões mais confusas do espaço. Esse resultado reflete a limitação de um MLP raso: apesar de capturar padrões não lineares, a fronteira de decisão ainda não é suficientemente expressiva para classes com forte sobreposição.

## Exercício 4 — MLP mais profundo (≥2 camadas ocultas)

In [ ]:
# 4 -> 32 -> 16 -> 3 (tanh nas ocultas, softmax na saída)
mlp_deep = MLP(layer_sizes=[X_train.shape[1], 32, 16, 3],
               activations=['tanh', 'tanh', 'softmax'],
               loss='ce', lr=0.05, l2=1e-4, seed=42)

Y_train = one_hot(y_train, 3)
Y_test  = one_hot(y_test, 3)

mlp_deep.fit(X_train, Y_train, epochs=500, verbose=True)
y_pred = mlp_deep.predict(X_test)
acc = (y_pred == y_test).mean()
print(f"Acurácia (teste, 2 ocultas): {acc:.3f}")


In [ ]:
plt.plot(mlp_deep.loss_hist); plt.xlabel("época"); plt.ylabel("loss (CE)")
plt.title("Curva de treino — MLP profundo"); plt.show()

Z = PCA(n_components=2, random_state=0).fit_transform(X_test)
y_hat = mlp_mc.predict(X_test)
for c in np.unique(y_test):
    pts = Z[y_test==c]
    plt.scatter(pts[:,0], pts[:,1], s=12, label=f"true {c}")
plt.scatter(Z[:,0], Z[:,1], c=y_hat, s=8, cmap="tab10", alpha=0.25, label="pred")
plt.legend(); plt.title("Teste (PCA 2D): rótulo real vs. predito"); plt.show()


Com duas camadas ocultas, a perda caiu de **1.29 → 0.47** em 500 épocas, num ritmo mais acelerado e contínuo que no caso anterior. A curva de treino mostra melhora clara na capacidade de ajuste.

No teste, a acurácia subiu para **80,7%**, superando o modelo raso. A projeção em PCA também evidencia maior alinhamento entre rótulos reais e predições, embora ainda haja sobreposição entre classes vizinhas. Esse resultado confirma o ganho de expressividade ao aumentar a profundidade: o MLP profundo consegue capturar fronteiras de decisão mais complexas e se adapta melhor ao problema.